In [240]:
import pandas as pd
import re
FICHEROS_CSV = [
            'data_dictionary', 'order_details',
            'orders', 'pizza_types', 'pizzas']

In [241]:
'''i = 3

df = pd.read_csv(FICHEROS_CSV[i] + '.csv', sep=',', encoding = 'cp1252')
string = df.dtypes.to_string()
mensaje = '\nEl fichero ' + FICHEROS_CSV[i]
mensaje += '.csv contiene las siguientes columnas:\n'
mensaje += '\n' + string +'\n'
file = open('analisis_datos.txt', 'w')
file.write(mensaje)
file.close()'''

"i = 3\n\ndf = pd.read_csv(FICHEROS_CSV[i] + '.csv', sep=',', encoding = 'cp1252')\nstring = df.dtypes.to_string()\nmensaje = '\nEl fichero ' + FICHEROS_CSV[i]\nmensaje += '.csv contiene las siguientes columnas:\n'\nmensaje += '\n' + string +'\n'\nfile = open('analisis_datos.txt', 'w')\nfile.write(mensaje)\nfile.close()"

In [242]:
'''string = df.dtypes.to_dict()
for key in string.keys():
    string[key] = str(string[key])
print(string)
dicc = {FICHEROS_CSV[i] + '.csv':string}
print(dicc)
df2 = pd.DataFrame(dicc)
df2'''

"string = df.dtypes.to_dict()\nfor key in string.keys():\n    string[key] = str(string[key])\nprint(string)\ndicc = {FICHEROS_CSV[i] + '.csv':string}\nprint(dicc)\ndf2 = pd.DataFrame(dicc)\ndf2"

In [243]:
'''file = open('analisis_datos.txt', 'w')
file.close()

for archivo in FICHEROS_CSV:

    df = pd.read_csv(archivo + '.csv', sep=',', encoding = 'cp1252')

    mensaje = '\nEl fichero ' + archivo
    mensaje += '.csv contiene las siguientes columnas:\n\n'

    for colum in df.columns:
        tmp = colum
        tmp += '    ' + str(df[colum].dtype)
        tmp += '    Valores Null/Nan = ' + str(df[colum].isnull().sum())
        mensaje += tmp + '\n'

    file = open('analisis_datos.txt', 'a')
    file.write(mensaje)
    file.close()
'''

"file = open('analisis_datos.txt', 'w')\nfile.close()\n\nfor archivo in FICHEROS_CSV:\n\n    df = pd.read_csv(archivo + '.csv', sep=',', encoding = 'cp1252')\n\n    mensaje = '\nEl fichero ' + archivo\n    mensaje += '.csv contiene las siguientes columnas:\n\n'\n\n    for colum in df.columns:\n        tmp = colum\n        tmp += '    ' + str(df[colum].dtype)\n        tmp += '    Valores Null/Nan = ' + str(df[colum].isnull().sum())\n        mensaje += tmp + '\n'\n\n    file = open('analisis_datos.txt', 'a')\n    file.write(mensaje)\n    file.close()\n"

In [244]:

df_orders = pd.read_csv('orders.csv')
ini = pd.Timestamp('26 Dec 2014')
fin = pd.Timestamp('1 Jan 2016')
SEMANAS = pd.date_range(start=ini, end=fin, freq='W-Mon')
prueba = pd.Timestamp('2015-01-11')
principio = SEMANAS[0]
fin = principio + pd.Timedelta('7 Day')
FECHAS = pd.date_range(principio, fin)[:7]


In [245]:
def filtrar_fechas(fecha:str):
    fecha = pd.to_datetime(fecha, format='%d/%m/%Y')
    ini = pd.Timestamp('1 Jan 2015')
    fin = pd.Timestamp('1 Feb 2015')
    fechas = pd.date_range(start=ini, end=fin, normalize=True)
    fechas = fechas[:len(fechas) - 1]
    if fecha in fechas:
        return fecha
    else:
        return float('nan')

In [246]:
df_orders['date'] = df_orders['date'].apply(filtrar_fechas)
df_dates = df_orders.dropna()
df_dates

,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40
2,3,2015-01-01,12:12:28
3,4,2015-01-01,12:16:31
4,5,2015-01-01,12:21:30
...,...,...,...
1840,1841,2015-01-31,21:01:18
1841,1842,2015-01-31,21:07:17
1842,1843,2015-01-31,21:28:07
1843,1844,2015-01-31,22:09:40


In [247]:
df_pedidos = pd.read_csv('order_details.csv', sep=',', encoding='cp1252')
df_tipos = pd.read_csv('pizzas.csv', sep=',', encoding='cp1252')

df_pedidos = df_pedidos.merge(df_dates, how='inner', left_on='order_id', right_on='order_id')
df_pizzas = df_pedidos.groupby(df_pedidos['pizza_id']).sum(numeric_only=True)
df_pizzas

,order_details_id,order_id,quantity
pizza_id,,,
bbq_ckn_l,200722,89012,96
bbq_ckn_m,163853,72642,78
bbq_ckn_s,74054,32797,37
big_meat_s,307461,136159,150
brie_carre_s,79803,35353,35
...,...,...,...
the_greek_xl,104912,46446,50
the_greek_xxl,7012,3109,2
veggie_veg_l,80899,35837,38


In [248]:
df_pizzas_total = df_pizzas.merge(df_tipos, how='inner', left_on='pizza_id', right_on='pizza_id')
df_pizzas_total = df_pizzas_total[['pizza_type_id', 'size', 'quantity']]
df_pizzas_total

,pizza_type_id,size,quantity
0,bbq_ckn,L,96
1,bbq_ckn,M,78
2,bbq_ckn,S,37
3,big_meat,S,150
4,brie_carre,S,35
...,...,...,...
86,the_greek,XL,50
87,the_greek,XXL,2
88,veggie_veg,L,38
89,veggie_veg,M,65


In [249]:
df_ingredientes_pizzas = pd.read_csv('pizza_types.csv', sep=',', encoding='cp1252')
df_ingredientes_pizzas = df_ingredientes_pizzas[['pizza_type_id', 'ingredients']]
df_pizzas_final = df_pizzas_total.merge(df_ingredientes_pizzas, how='inner', left_on='pizza_type_id', right_on='pizza_type_id')


In [250]:
def str_to_list(string: str):

    if isinstance(string, str):
        salsa = 0
        queso = 0
        if not re.search('Sauce', string):
            salsa = 1
        if not re.search('Mozzarella Cheese', string):
            queso = 1
        string = string.split(', ')
        if salsa:
            string.append('Tomato Sauce')
        if queso:
            string.append('Mozarrella Cheese')
    return string

df_pizzas_final['ingredients'] = df_pizzas_final['ingredients'].apply(str_to_list)


    for i in df_final.index:

        # Recogemos los datos del dataframe que reutilizaremos
        pizza = df_final['pizza_type_id'][i]
        size = df_final['size'][i]
        cantidad = df_final['quantity'][i]

        '''dicc['pizza_type_id'].append(pizza)
        dicc['size'].append(size)'''
        dicc['quantity'].append(cantidad)

        # Y ahora, por cada ingrediente, hemos de ver si está en la pizza
        for ingrediente in ingredientes:

            # Si está, añadiremos a la lista la proporción del ingrediente
            # segun el tamaño
            if ingrediente in df_final['ingredients'][i]:

                dicc[ingrediente] += [cantidad * sizes[size]]

            # Y si no está, fijaremos su valor a 0
            elif ingrediente not in df_final['ingredients'][i]:

                dicc[ingrediente] += [0]

    # Por último, guardaremos todo en un dataframe
    df_final = pd.DataFrame(dicc)

    return df_final.sum(numeric_only=True)

In [251]:
dicc = {'pizza_type_id': [], 'size': [], 'Pedidos esperados': []}
total = {}
ingredientes = []
for i in df_pizzas_final.index:
    for j in range(len(df_pizzas_final['ingredients'][i])):
        if df_pizzas_final['ingredients'][i][j] not in ingredientes:
            ingredientes.append(df_pizzas_final['ingredients'][i][j])
            dicc[df_pizzas_final['ingredients'][i][j]] = []
            total[df_pizzas_final['ingredients'][i][j]] = 0
sizes = {'S': 1, 'M': 1.5, 'L': 2,'XL': 2.5, 'XXL': 3}
# print(ingredientes)
for i in df_pizzas_final.index:
    pizza = df_pizzas_final['pizza_type_id'][i]
    size = df_pizzas_final['size'][i]
    cantidad = df_pizzas_final['quantity'][i]
    dicc['pizza_type_id'].append(pizza)
    dicc['size'].append(size)
    dicc['Pedidos esperados'].append(cantidad)
    for ingrediente in ingredientes:
        if ingrediente in df_pizzas_final['ingredients'][i]:
            
            dicc[ingrediente] += [sizes[size]]
            total[ingrediente] += cantidad * sizes[size]
            
        elif ingrediente not in df_pizzas_final['ingredients'][i]:
            dicc[ingrediente] += [0]

dicc['pizza_type_id'].append('Total')
dicc['size'].append('Total')
dicc['Pedidos esperados'].append(df_pizzas_final['quantity'].sum())
print(total)
for ingrediente in ingredientes:

    dicc[ingrediente] += [total[ingrediente]]
df_tmp = pd.DataFrame(dicc)


{'Barbecued Chicken': 346.0, 'Red Peppers': 2227.0, 'Green Peppers': 708.5, 'Tomatoes': 3740.5, 'Red Onions': 2677.0, 'Barbecue Sauce': 346.0, 'Mozarrella Cheese': 5090.0, 'Bacon': 420.5, 'Pepperoni': 1267.5, 'Italian Sausage': 150, 'Chorizo Sausage': 150, 'Tomato Sauce': 4951.5, 'Brie Carre Cheese': 35, 'Prosciutto': 35, 'Caramelized Onions': 35, 'Pears': 35, 'Thyme': 35, 'Garlic': 3956.0, '‘Nduja Salami': 109.0, 'Pancetta': 109.0, 'Friggitello Peppers': 109.0, 'Chicken': 1135.5, 'Artichoke': 314.0, 'Spinach': 1279.0, 'Jalapeno Peppers': 770.5, 'Fontina Cheese': 434.0, 'Gouda Cheese': 314.0, 'Mushrooms': 1171.0, 'Asiago Cheese': 468.0, 'Alfredo Sauce': 131.0, 'Pesto Sauce': 382.5, 'Mozzarella Cheese': 1516.5, 'Provolone Cheese': 276, 'Smoked Gouda Cheese': 276, 'Romano Cheese': 276, 'Blue Cheese': 276, 'Ricotta Cheese': 289.5, 'Gorgonzola Piccante Cheese': 289.5, 'Parmigiano Reggiano Cheese': 289.5, 'Green Olives': 857.5, 'Feta Cheese': 636.0, 'Sliced Ham': 269.5, 'Pineapple': 608.5, 

In [252]:
tmp = 0
for i in df_tmp.index:
    if i != 91:
        tmp += df_tmp['Tomato Sauce'][i]* df_tmp['Pedidos esperados'][i]

In [253]:
df_tmp

,pizza_type_id,size,Pedidos esperados,Barbecued Chicken,Red Peppers,Green Peppers,Tomatoes,Red Onions,Barbecue Sauce,Mozarrella Cheese,...,Genoa Salami,Prosciutto di San Daniele,Arugula,Coarse Sicilian Salami,Luganega Sausage,Onions,Soppressata Salami,Peperoncini verdi,Thai Sweet Chilli Sauce,Beef Chuck Roast
0,bbq_ckn,L,96,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,bbq_ckn,M,78,1.5,1.5,1.5,1.5,1.5,1.5,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bbq_ckn,S,37,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,big_meat,S,150,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,brie_carre,S,35,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,the_greek,XXL,2,0.0,0.0,0.0,3.0,3.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
88,veggie_veg,L,38,0.0,2.0,2.0,2.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89,veggie_veg,M,65,0.0,1.5,1.5,1.5,1.5,0.0,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,veggie_veg,S,40,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [254]:
df_ingredientes = df_tmp.tail(1)

df_ingredientes.pop('pizza_type_id')
df_ingredientes.pop('size')
df_ingredientes

,Pedidos esperados,Barbecued Chicken,Red Peppers,Green Peppers,Tomatoes,Red Onions,Barbecue Sauce,Mozarrella Cheese,Bacon,Pepperoni,...,Genoa Salami,Prosciutto di San Daniele,Arugula,Coarse Sicilian Salami,Luganega Sausage,Onions,Soppressata Salami,Peperoncini verdi,Thai Sweet Chilli Sauce,Beef Chuck Roast
91,4232,346.0,2227.0,708.5,3740.5,2677.0,346.0,5090.0,420.5,1267.5,...,217.0,200.5,200.5,272.0,272.0,272.0,120.0,256.5,339.0,240.5


In [255]:
ERROR = 4


def aproximar_numero(numero: float):
    '''
    Aproxima un número, añadiéndole un margen de error
    '''

    global ERROR

    return int(numero) + ERROR

In [256]:
df_ingredientes

,Pedidos esperados,Barbecued Chicken,Red Peppers,Green Peppers,Tomatoes,Red Onions,Barbecue Sauce,Mozarrella Cheese,Bacon,Pepperoni,...,Genoa Salami,Prosciutto di San Daniele,Arugula,Coarse Sicilian Salami,Luganega Sausage,Onions,Soppressata Salami,Peperoncini verdi,Thai Sweet Chilli Sauce,Beef Chuck Roast
91,4232,346.0,2227.0,708.5,3740.5,2677.0,346.0,5090.0,420.5,1267.5,...,217.0,200.5,200.5,272.0,272.0,272.0,120.0,256.5,339.0,240.5


In [260]:
df_ingredientes = df_ingredientes.apply(aproximar_numero)
type(df_ingredientes)

pandas.core.series.Series

In [271]:
df_ingredientes = pd.DataFrame(df_ingredientes)
df_ingredientes = df_ingredientes.rename(columns={0: 'Nº esperado'})
df_ingredientes

,Nº esperado
Pedidos esperados,4240
Barbecued Chicken,354
Red Peppers,2235
Green Peppers,716
Tomatoes,3748
...,...
Onions,280
Soppressata Salami,128
Peperoncini verdi,264
Thai Sweet Chilli Sauce,347


In [272]:
pd.read_csv('patata.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'patata.csv'